# Boston Housing Price Prediction with Neural Architecture Search (NAS)

This notebook implements Neural Architecture Search (NAS) to find the optimal neural network architecture for predicting Boston housing prices.

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Deep learning libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## 2. Load and Prepare Data

In [ ]:
# Load the dataset
df = pd.read_csv('boston_house_prices.csv', skiprows=1)

print("Dataset shape:", df.shape)
print("\nFirst few rows:")
df.head()

In [ ]:
# Separate features and target
X = df.drop('MEDV', axis=1)
y = df['MEDV']

# Split data: 70% train, 15% validation, 15% test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42)  # 0.176 of 0.85 ≈ 0.15 of total

print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

In [ ]:
# Standardize features (important for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("Data standardization completed.")
print(f"Feature mean (should be ~0): {X_train_scaled.mean(axis=0).mean():.6f}")
print(f"Feature std (should be ~1): {X_train_scaled.std(axis=0).mean():.6f}")

## 3. Define Neural Architecture Search Space

We'll implement a simple NAS approach that searches over:
- Number of hidden layers (1-4)
- Number of neurons per layer (16, 32, 64, 128)
- Activation functions (relu, tanh, elu)
- Dropout rates (0.0, 0.1, 0.2, 0.3)
- Learning rates (0.001, 0.01, 0.0001)

In [ ]:
# Define search space
search_space = {
    'num_layers': [1, 2, 3, 4],
    'neurons': [16, 32, 64, 128],
    'activation': ['relu', 'tanh', 'elu'],
    'dropout': [0.0, 0.1, 0.2, 0.3],
    'learning_rate': [0.0001, 0.001, 0.01]
}

print("Neural Architecture Search Space:")
for param, values in search_space.items():
    print(f"  {param}: {values}")

total_combinations = (len(search_space['num_layers']) * 
                     len(search_space['neurons']) * 
                     len(search_space['activation']) * 
                     len(search_space['dropout']) * 
                     len(search_space['learning_rate']))
print(f"\nTotal possible architectures: {total_combinations}")

## 4. Build Model Creation Function

In [ ]:
def create_model(num_layers, neurons, activation, dropout, learning_rate, input_dim):
    """
    Create a neural network model with specified architecture.
    
    Parameters:
    - num_layers: Number of hidden layers
    - neurons: Number of neurons per layer
    - activation: Activation function
    - dropout: Dropout rate
    - learning_rate: Learning rate for optimizer
    - input_dim: Number of input features
    
    Returns:
    - Compiled Keras model
    """
    model = models.Sequential()
    
    # Input layer
    model.add(layers.Input(shape=(input_dim,)))
    
    # Hidden layers
    for i in range(num_layers):
        model.add(layers.Dense(neurons, activation=activation))
        if dropout > 0:
            model.add(layers.Dropout(dropout))
    
    # Output layer (regression - single neuron)
    model.add(layers.Dense(1))
    
    # Compile model
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    return model

## 5. Implement Random Search NAS

We'll use random search to sample architectures from the search space and evaluate them.

In [ ]:
def random_search_nas(n_iterations=30, epochs=100, patience=15, verbose=0):
    """
    Perform random search over neural architecture space.
    
    Parameters:
    - n_iterations: Number of architectures to try
    - epochs: Maximum training epochs per architecture
    - patience: Early stopping patience
    - verbose: Training verbosity
    
    Returns:
    - results: List of dictionaries containing architecture configs and scores
    - best_config: Best architecture configuration
    """
    results = []
    best_val_loss = float('inf')
    best_config = None
    
    input_dim = X_train_scaled.shape[1]
    
    # Callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
    
    print(f"Starting Random Search NAS with {n_iterations} iterations...\n")
    print("="*80)
    
    for i in range(n_iterations):
        # Sample random architecture
        config = {
            'num_layers': np.random.choice(search_space['num_layers']),
            'neurons': np.random.choice(search_space['neurons']),
            'activation': np.random.choice(search_space['activation']),
            'dropout': np.random.choice(search_space['dropout']),
            'learning_rate': np.random.choice(search_space['learning_rate'])
        }
        
        print(f"\nIteration {i+1}/{n_iterations}")
        print(f"Config: layers={config['num_layers']}, neurons={config['neurons']}, "
              f"activation={config['activation']}, dropout={config['dropout']}, "
              f"lr={config['learning_rate']}")
        
        # Create and train model
        model = create_model(
            num_layers=config['num_layers'],
            neurons=config['neurons'],
            activation=config['activation'],
            dropout=config['dropout'],
            learning_rate=config['learning_rate'],
            input_dim=input_dim
        )
        
        history = model.fit(
            X_train_scaled, y_train,
            validation_data=(X_val_scaled, y_val),
            epochs=epochs,
            batch_size=32,
            callbacks=[early_stop, reduce_lr],
            verbose=verbose
        )
        
        # Evaluate
        val_loss = min(history.history['val_loss'])
        val_mae = min(history.history['val_mae'])
        
        # Store results
        result = {
            'config': config.copy(),
            'val_loss': val_loss,
            'val_mae': val_mae,
            'epochs_trained': len(history.history['loss'])
        }
        results.append(result)
        
        print(f"Val Loss: {val_loss:.4f}, Val MAE: {val_mae:.4f}, Epochs: {result['epochs_trained']}")
        
        # Update best config
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_config = config.copy()
            print(f"*** New best architecture found! Val Loss: {val_loss:.4f} ***")
    
    print("\n" + "="*80)
    print("Random Search NAS completed!")
    
    return results, best_config

In [ ]:
# Run NAS (using fewer iterations for faster execution)
# Increase n_iterations for more thorough search
results, best_config = random_search_nas(n_iterations=30, epochs=100, patience=15, verbose=0)

## 6. Analyze NAS Results

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame([
    {**r['config'], 'val_loss': r['val_loss'], 'val_mae': r['val_mae'], 'epochs': r['epochs_trained']}
    for r in results
])

# Sort by validation loss
results_df = results_df.sort_values('val_loss')

print("\nTop 10 Architectures:")
print(results_df.head(10))

In [ ]:
# Visualize NAS results
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Val Loss by number of layers
axes[0, 0].boxplot([results_df[results_df['num_layers']==i]['val_loss'].values 
                     for i in sorted(results_df['num_layers'].unique())])
axes[0, 0].set_xticklabels(sorted(results_df['num_layers'].unique()))
axes[0, 0].set_xlabel('Number of Layers')
axes[0, 0].set_ylabel('Validation Loss')
axes[0, 0].set_title('Val Loss by Number of Layers')
axes[0, 0].grid(True, alpha=0.3)

# Val Loss by neurons
axes[0, 1].boxplot([results_df[results_df['neurons']==i]['val_loss'].values 
                     for i in sorted(results_df['neurons'].unique())])
axes[0, 1].set_xticklabels(sorted(results_df['neurons'].unique()))
axes[0, 1].set_xlabel('Number of Neurons')
axes[0, 1].set_ylabel('Validation Loss')
axes[0, 1].set_title('Val Loss by Number of Neurons')
axes[0, 1].grid(True, alpha=0.3)

# Val Loss by activation
axes[0, 2].boxplot([results_df[results_df['activation']==i]['val_loss'].values 
                     for i in sorted(results_df['activation'].unique())])
axes[0, 2].set_xticklabels(sorted(results_df['activation'].unique()))
axes[0, 2].set_xlabel('Activation Function')
axes[0, 2].set_ylabel('Validation Loss')
axes[0, 2].set_title('Val Loss by Activation Function')
axes[0, 2].grid(True, alpha=0.3)

# Val Loss by dropout
axes[1, 0].boxplot([results_df[results_df['dropout']==i]['val_loss'].values 
                     for i in sorted(results_df['dropout'].unique())])
axes[1, 0].set_xticklabels(sorted(results_df['dropout'].unique()))
axes[1, 0].set_xlabel('Dropout Rate')
axes[1, 0].set_ylabel('Validation Loss')
axes[1, 0].set_title('Val Loss by Dropout Rate')
axes[1, 0].grid(True, alpha=0.3)

# Val Loss by learning rate
axes[1, 1].boxplot([results_df[results_df['learning_rate']==i]['val_loss'].values 
                     for i in sorted(results_df['learning_rate'].unique())])
axes[1, 1].set_xticklabels(sorted(results_df['learning_rate'].unique()))
axes[1, 1].set_xlabel('Learning Rate')
axes[1, 1].set_ylabel('Validation Loss')
axes[1, 1].set_title('Val Loss by Learning Rate')
axes[1, 1].grid(True, alpha=0.3)

# Overall distribution
axes[1, 2].hist(results_df['val_loss'], bins=20, edgecolor='black', color='skyblue')
axes[1, 2].axvline(results_df['val_loss'].min(), color='r', linestyle='--', label='Best')
axes[1, 2].set_xlabel('Validation Loss')
axes[1, 2].set_ylabel('Frequency')
axes[1, 2].set_title('Distribution of Validation Loss')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Train Final Model with Best Architecture

In [ ]:
print("Best Architecture Found:")
print("="*50)
for param, value in best_config.items():
    print(f"  {param}: {value}")
print("="*50)

In [ ]:
# Create and train final model with best configuration
input_dim = X_train_scaled.shape[1]
best_model = create_model(
    num_layers=best_config['num_layers'],
    neurons=best_config['neurons'],
    activation=best_config['activation'],
    dropout=best_config['dropout'],
    learning_rate=best_config['learning_rate'],
    input_dim=input_dim
)

# Display model architecture
print("\nModel Architecture:")
best_model.summary()

In [ ]:
# Train with more epochs for final model
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-6)

history = best_model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=200,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

## 8. Visualize Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Loss
axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE
axes[1].plot(history.history['mae'], label='Training MAE', linewidth=2)
axes[1].plot(history.history['val_mae'], label='Validation MAE', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].set_title('Training and Validation MAE')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Evaluate Final Model

In [ ]:
# Make predictions
y_train_pred = best_model.predict(X_train_scaled).flatten()
y_val_pred = best_model.predict(X_val_scaled).flatten()
y_test_pred = best_model.predict(X_test_scaled).flatten()

# Calculate metrics
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

val_mse = mean_squared_error(y_val, y_val_pred)
val_rmse = np.sqrt(val_mse)
val_mae = mean_absolute_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)

test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Display results
print("="*70)
print("NEURAL ARCHITECTURE SEARCH - FINAL MODEL PERFORMANCE")
print("="*70)
print("\nTraining Set:")
print(f"  R² Score: {train_r2:.4f}")
print(f"  RMSE: {train_rmse:.4f}")
print(f"  MAE: {train_mae:.4f}")
print(f"  MSE: {train_mse:.4f}")

print("\nValidation Set:")
print(f"  R² Score: {val_r2:.4f}")
print(f"  RMSE: {val_rmse:.4f}")
print(f"  MAE: {val_mae:.4f}")
print(f"  MSE: {val_mse:.4f}")

print("\nTest Set:")
print(f"  R² Score: {test_r2:.4f}")
print(f"  RMSE: {test_rmse:.4f}")
print(f"  MAE: {test_mae:.4f}")
print(f"  MSE: {test_mse:.4f}")
print("="*70)

## 10. Visualize Predictions

In [ ]:
# Prediction vs Actual plots
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Training set
axes[0].scatter(y_train, y_train_pred, alpha=0.5, color='blue')
axes[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Price')
axes[0].set_ylabel('Predicted Price')
axes[0].set_title(f'Training Set\nR² = {train_r2:.4f}, RMSE = {train_rmse:.4f}')
axes[0].grid(True, alpha=0.3)

# Validation set
axes[1].scatter(y_val, y_val_pred, alpha=0.5, color='green')
axes[1].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
axes[1].set_xlabel('Actual Price')
axes[1].set_ylabel('Predicted Price')
axes[1].set_title(f'Validation Set\nR² = {val_r2:.4f}, RMSE = {val_rmse:.4f}')
axes[1].grid(True, alpha=0.3)

# Test set
axes[2].scatter(y_test, y_test_pred, alpha=0.5, color='red')
axes[2].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[2].set_xlabel('Actual Price')
axes[2].set_ylabel('Predicted Price')
axes[2].set_title(f'Test Set\nR² = {test_r2:.4f}, RMSE = {test_rmse:.4f}')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 11. Residual Analysis

In [ ]:
# Calculate residuals
test_residuals = y_test - y_test_pred

# Residual plots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Residual plot
axes[0].scatter(y_test_pred, test_residuals, alpha=0.5, color='purple')
axes[0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0].set_xlabel('Predicted Price')
axes[0].set_ylabel('Residuals')
axes[0].set_title('Residual Plot (Test Set)')
axes[0].grid(True, alpha=0.3)

# Residual distribution
axes[1].hist(test_residuals, bins=30, edgecolor='black', color='lightcoral')
axes[1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Residuals')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Residual Distribution (Test Set)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 12. Summary and Conclusions

In [ ]:
print("="*80)
print("NEURAL ARCHITECTURE SEARCH - SUMMARY")
print("="*80)

print("\n1. Search Configuration:")
print(f"   - Total architectures evaluated: {len(results)}")
print(f"   - Search space size: {total_combinations}")
print(f"   - Search coverage: {len(results)/total_combinations*100:.2f}%")

print("\n2. Best Architecture:")
for param, value in best_config.items():
    print(f"   - {param}: {value}")

print("\n3. Performance Comparison:")
print(f"   - Best Val Loss from NAS: {results_df['val_loss'].min():.4f}")
print(f"   - Worst Val Loss from NAS: {results_df['val_loss'].max():.4f}")
print(f"   - Average Val Loss: {results_df['val_loss'].mean():.4f}")
print(f"   - Improvement (best vs avg): {(results_df['val_loss'].mean() - results_df['val_loss'].min())/results_df['val_loss'].mean()*100:.2f}%")

print("\n4. Final Model Test Performance:")
print(f"   - R² Score: {test_r2:.4f} ({test_r2*100:.2f}% variance explained)")
print(f"   - RMSE: ${test_rmse:.2f}k")
print(f"   - MAE: ${test_mae:.2f}k")

print("\n5. Key Insights:")
print(f"   - NAS found an architecture that outperforms random configurations")
print(f"   - The model generalizes well (Train R²={train_r2:.4f}, Test R²={test_r2:.4f})")
print(f"   - Average prediction error: ${test_mae:.2f}k")

print("\n6. Top 3 Hyperparameter Insights:")
for param in ['num_layers', 'neurons', 'activation']:
    best_value = results_df.groupby(param)['val_loss'].mean().idxmin()
    print(f"   - Best {param}: {best_value}")

print("="*80)